<a href="https://colab.research.google.com/github/omkarpat/conversational-movie-recommender/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/Courses/CSE243/Baseline
#%cd Baseline/

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline


In [ ]:
!ls

batch_loaders	       movielens	 sequence_generator.py
beam_search.py	       __pycache__	 test_params.py
config.py	       README.md	 train_autorec.py
generate_responses.py  redial		 train_recommender.py
ml-latest.zip	       requirements.txt  train_sentiment_analysis.py
models		       scripts		 utils.py


In [ ]:
!pip3 install -r requirements.txt
!python3 -m nltk.downloader punkt

     |████████████████████████████████| 519.5MB 29kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!wget -O redial/redial_dataset.zip https://github.com/ReDialData/website/raw/data/redial_dataset.zip
!wget -O movielens/ml-latest.zip http://files.grouplens.org/datasets/movielens/ml-latest.zip
# split ReDial data
!python3 scripts/split-redial.py redial/
!mv redial/test_data.jsonl redial/test_data
# split Movielens data
!python3 scripts/split-movielens.py movielens/

--2020-11-03 20:07:40--  https://github.com/ReDialData/website/raw/data/redial_dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ReDialData/website/data/redial_dataset.zip [following]
--2020-11-03 20:07:40--  https://raw.githubusercontent.com/ReDialData/website/data/redial_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5765261 (5.5M) [application/zip]
Saving to: ‘redial/redial_dataset.zip’

redial/redial_datas 100%[===================>]   5.50M  --.-KB/s    in 0.1s    

2020-11-03 20:07:40 (55.2 MB/s) - ‘redial/redial_dataset.zip’ saved [5765261/5765261]

--2020-11-03 20:07:40-

In [ ]:
!python scripts/match_movies.py --redial_movies_path=redial/movies_with_mentions.csv --ml_movies_path=movielens/ml-latest/movies.csv --destination=redial/movies_merged.csv

loaded 58098 movies from movielens/ml-latest/movies.csv
loaded 6924 movies from redial/movies_with_mentions.csv
100% 6924/6924 [01:36<00:00, 72.06it/s]
Over 6924 movies mentioned in ReDial, 5178 of them are perfectly matched, 1746 of them have no match in movielens


In [ ]:
%cd models/
!mkdir GenSen embeddings
!wget -O GenSen/nli_large_vocab.pkl https://genseniclr2018.blob.core.windows.net/models/nli_large_vocab.pkl
!wget -O GenSen/nli_large.model https://genseniclr2018.blob.core.windows.net/models/nli_large.model
%cd embeddings
!wget https://raw.githubusercontent.com/Maluuba/gensen/master/data/embedding/glove2h5.py
!wget https://github.com/Maluuba/gensen/raw/master/data/embedding/glove2h5.sh
!sh glove2h5.sh
% cd ..

/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline/models
mkdir: cannot create directory ‘GenSen’: File exists
mkdir: cannot create directory ‘embeddings’: File exists
--2020-11-04 03:50:51--  https://genseniclr2018.blob.core.windows.net/models/nli_large_vocab.pkl
Resolving genseniclr2018.blob.core.windows.net (genseniclr2018.blob.core.windows.net)... 52.239.154.100
Connecting to genseniclr2018.blob.core.windows.net (genseniclr2018.blob.core.windows.net)|52.239.154.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3270098 (3.1M) [application/octet-stream]
Saving to: ‘GenSen/nli_large_vocab.pkl’

GenSen/nli_large_vo 100%[===================>]   3.12M  15.9MB/s    in 0.2s    

2020-11-04 03:50:51 (15.9 MB/s) - ‘GenSen/nli_large_vocab.pkl’ saved [3270098/3270098]

--2020-11-04 03:50:52--  https://genseniclr2018.blob.core.windows.net/models/nli_large.model
Resolving genseniclr2018.blob.core.windows.net (genseniclr2018.blob.

In [ ]:
cd ../..

/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline


In [ ]:
!python train_sentiment_analysis.py

Saving in models/sentiment_analysis with parameters : {'hrnn_params': {'use_movie_occurrences': 'word', 'sentence_encoder_hidden_size': 512, 'conversation_encoder_hidden_size': 512, 'sentence_encoder_num_layers': 2, 'conversation_encoder_num_layers': 2, 'use_dropout': 0.4}}, {'learning_rate': 0.001, 'batch_size': 16, 'nb_epochs': 50, 'patience': 5, 'weight_decay': 0, 'use_class_weights': True, 'cut_dialogues': -1, 'targets': 'suggested seen liked'}
loaded 59946 movies from redial/movies_merged.csv
6924 movies
Loading and processing data
Loading vocabulary from redial/vocabulary.p
Traceback (most recent call last):
  File "train_sentiment_analysis.py", line 261, in <module>
    explore_params(params_seq, baseline=False)
  File "train_sentiment_analysis.py", line 194, in explore_params
    batch_size=train_params['batch_size']
  File "/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline/batch_loaders/batch_loader.py", line 110, in __init__
    self._g

In [ ]:
!python train_autorec.py

Saving in models/autorec with parameters : {'layer_sizes': [1000], 'f': 'sigmoid', 'g': 'sigmoid'}, {'learning_rate': 0.001, 'batch_size': 64, 'nb_epochs': 50, 'patience': 5, 'batch_input': 'random_noise', 'max_num_inputs': 10000000000.0}
Traceback (most recent call last):
  File "train_autorec.py", line 204, in <module>
    explore_params(params_seq=params, data="db_pretrain")
  File "train_autorec.py", line 158, in explore_params
    ratings01=True
  File "/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline/batch_loaders/ml_batch_loader.py", line 80, in __init__
    self.load_data()
  File "/content/gdrive/.shortcut-targets-by-id/1vjVd4Rk4nmCTqyygyyGCj589eDwzuLUI/CSE243/Baseline/batch_loaders/ml_batch_loader.py", line 88, in load_data
    self.n_movies = np.max(self.id2index.values()) + 1
TypeError: unsupported operand type(s) for +: 'dict_values' and 'int'
